In [ ]:
from whales import engine_futures, db_path_futures, engine
from sqlalchemy.orm import Session
from sqlalchemy import text
from models.histpricemodel import HistPrice
import pandas as pd

from datetime import datetime

In [ ]:

with Session(engine_futures) as session:
    _query = text("SELECT * FROM histprice;")
    #resultados = session.execute(_query).all()
    dados = pd.read_sql(sql=_query, con=engine_futures)
    #print(dados[:9])

In [ ]:
dados

In [ ]:
edited_data = dados.copy()

In [ ]:
edited_data['opentime'] = [datetime.fromtimestamp(time/1000) for time in edited_data['opentime']]
edited_data['closetime'] = [datetime.fromtimestamp(time/1000) for time in edited_data['closetime']]

In [ ]:
edited_data

In [ ]:
dataset = edited_data.query("pair == 'ADAUSDT'")

In [ ]:
dataset.to_csv('data/ada_prices_1m.csv')

#### Loading dataset from csv file...

In [ ]:
dataset = pd.read_csv('data/ada_prices_1m.csv', parse_dates=True, index_col='opentime')
dataset.sort_index(ascending=True, inplace=True)

In [ ]:
dataset

In [ ]:
dataset.drop(columns=['Unnamed: 0', 'id', 'pair', 'ignore'], axis=1, inplace=True)

In [ ]:
dataset.set_index('opentime', inplace=True)

In [ ]:
dataset.sort_index(ascending=True, inplace=True)

In [ ]:
dataset['diff'] = dataset['close'] - dataset['open']

In [ ]:
dataset

In [ ]:
from datetime import timedelta

In [ ]:
datetime.fromtimestamp(datetime.timestamp(dataset.index[0]) + (60 * 60))

In [ ]:
dataset.loc[dataset.index == datetime.fromtimestamp(datetime.timestamp(dataset.index[0]) + (60 * 60))]

In [ ]:
def get_targets(df: pd.DataFrame, period: int) -> pd.DataFrame:
    _new_df = pd.DataFrame()
    for i in range(df.__len__() - period):
        _tmp_df = df.loc[df.index == datetime.fromtimestamp(datetime.timestamp(df.index[i]) + (period * 60))]
        _new_df = pd.concat([_new_df, _tmp_df['close']], axis=0)
        
    return _new_df

In [ ]:
def get_targets_v2(df: pd.DataFrame, period: int) -> pd.DataFrame:
    _new_df = df.iloc[period:,:]
    return _new_df
    

In [ ]:
test = get_targets(dataset, 60),

In [ ]:
test2 = get_targets_v2(dataset, 60)

In [ ]:
test

In [ ]:
test2

In [ ]:
def include_targets(df_base: pd.DataFrame, df_futures: pd.DataFrame) -> pd.DataFrame:
    _tmp_prices = []
    for i in range(len(df_futures)):
        _tmp_prices.append(df_futures['close'][i])
    if len(_tmp_prices) < len(df_base):
        distance = len(df_base) - len(_tmp_prices)
        for item in range(distance):
            _tmp_prices.append(0)
    df_base['target_prices'] = _tmp_prices
    return df_base

In [ ]:
dataset_test = include_targets(dataset, test2)

In [ ]:
dataset = dataset_test[:-60]

In [ ]:
dataset['diff_target'] = dataset['target_prices'] - dataset['close']

In [ ]:
dataset

In [ ]:
def calculate_profit(close_price, diff_target):
    return ((diff_target * 100) / close_price)

In [ ]:
dataset['target_profit'] = calculate_profit(dataset['close'], dataset['diff_target'])

In [ ]:
dataset.reset_index(inplace=True)

In [ ]:
dataset.info()

In [ ]:
import matplotlib.pyplot as plt
from finta import TA

In [ ]:
dataset['RSI'] = TA.RSI(dataset, period=10)

In [ ]:
dataset['MSD'] = TA.MSD(dataset, period=10)

In [ ]:
dataset['OBV'] = TA.OBV(dataset)

In [ ]:
dataset['EMA'] = TA.EMA(dataset)

In [ ]:
dataset.shape

In [ ]:
bbands = TA.BBANDS(dataset, period=9, std_multiplier=1)

In [ ]:
bbands

In [ ]:
dataset['SDT_M'] = dataset['diff'].rolling(window=60).std()

In [ ]:
dataset['VFI'] = TA.VFI(dataset, period=60)

In [ ]:
pivot_points = TA.PIVOT(dataset)

In [ ]:
pivot_points

In [ ]:
dataset[61:-60].to_csv('data/ada_prices_with_more_features.csv', index=False)

In [ ]:
dataset.drop(columns=['opentime', 'open', 'high', 'low', 'close', 'closetime', 'quoteassetvolume', 'takerbuybaseassetvolume', 'takerbuyquoteassetvolume', 'target_prices', 'target_profit'], axis=1, inplace=True)

In [ ]:
dataset = dataset[61:-60]

In [ ]:
dataset['OBV'].fillna(0, inplace=True)
dataset['RSI'].fillna(0, inplace=True)
dataset['MSD'].fillna(0, inplace=True)

In [ ]:
#dataset.reset_index(inplace=True)

In [ ]:
#dataset.columns = ['close', 'volume', 'RSI', 'MSD', 'OBV', 'numberoftrades', 'diff', 'target_prices', 'diff_target', 'target_profit']

In [ ]:
dataset.reset_index(inplace=True)

In [ ]:
dataset.drop(columns=['index'], inplace=True)

In [ ]:
dataset

#### Normalization

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
dataset = pd.DataFrame(scaler.fit_transform(dataset), columns=dataset.columns)

In [ ]:
dataset

In [ ]:
dataset.to_csv('data/ada_prices_with_more_features_norm.csv', index=False)

In [ ]:
dataset = pd.read_csv('data/ada_prices_with_more_features_norm.csv')
dataset

### Train and Test Split

In [ ]:
tamanho_treino = int(len(dataset) * 0.80)
treino, teste = dataset[0:tamanho_treino], dataset[tamanho_treino:len(dataset)]

In [ ]:
# Mostrando quais dados servirão para treino e para teste
fig, ax = plt.subplots(1, figsize=(10, 5))
ax.plot(treino['diff_target'], label='treino', linewidth=1)
ax.plot(teste['diff_target'], label='teste', linewidth=1)
ax.set_ylabel('Função', fontsize=14)
ax.legend(loc='best', fontsize=16)

In [ ]:
# Mostrando quais dados servirão para treino e para teste
fig, ax = plt.subplots(1, figsize=(10, 5))
ax.plot(treino['diff'], label='treino', linewidth=1)
ax.plot(teste['diff'], label='teste', linewidth=1)
ax.set_ylabel('Função', fontsize=14)
ax.legend(loc='best', fontsize=16)

In [ ]:
dataset

In [ ]:
def price_direction(dataset: pd.DataFrame, direction_reference_col = 'diff_target') -> pd.DataFrame:
    index_target_col = dataset.columns.get_loc(direction_reference_col)
    _direction = []
    for i in range(len(dataset)):
        if dataset.iloc[i,index_target_col] > 0: _direction.append(1)
        elif dataset.iloc[i,index_target_col] < 0: _direction.append(-1)
        else: _direction.append(0)
    dataset['direction'] = _direction
    return dataset

In [ ]:
dataset = price_direction(dataset=dataset)

In [ ]:
dataset.drop(columns=['direction'], inplace=True)

In [ ]:
dataset

In [ ]:
treino

In [ ]:
teste

In [ ]:
tamanho_treino = int(len(dataset) * 0.80)
treino, teste = dataset[0:tamanho_treino], dataset[tamanho_treino:len(dataset)]

In [ ]:
import numpy as np

# Criando uma função para as janelas temporais. A saída será dois arrays, o primeiro contendo várias janelas do dataset, e o
# segundo contendo valores a serem previstos
def gera_dataset(dataset, tamanho_janela = 1, col_target_name = 'Close'):
    dataA, dataB = [], []
    for i in range(len(dataset)-tamanho_janela): # lembre-se que os datasets têm índices começando em zero
        a = dataset[i:(i+tamanho_janela)].copy()
        a.drop(col_target_name, axis=1, inplace=True)
        a = a.to_numpy()
        dataA.append(a)
        try:
            index_col = dataset.columns.get_loc(col_target_name)
            b = dataset.iloc[i+tamanho_janela,index_col].copy()
            #print(b)
            #b = b.to_numpy()
            
            dataB.append(b)
        except KeyError:
            break 
    return np.asarray(dataA, dtype=np.float32), np.asarray(dataB, dtype=np.float32)

In [ ]:
treino.shape, teste.shape

In [ ]:
teste.columns.get_loc('diff_target')

In [ ]:
# Aplicando a função nos datasets de treino e teste que já criamos:
tamanho_janela=60
x_treino, y_treino = gera_dataset(treino, tamanho_janela=tamanho_janela, col_target_name='diff_target')
x_teste, y_teste = gera_dataset(teste, tamanho_janela=tamanho_janela, col_target_name='diff_target')

In [ ]:
x_treino.shape, y_treino.shape

In [ ]:
x_teste.shape, y_teste.shape

In [ ]:
teste.shape, teste.head(), teste.iloc[0,-1], x_teste.shape, x_teste[0][0][-1], y_teste.shape

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import math
import warnings
warnings.filterwarnings('ignore')
import os
os.environ['CUDA_VISIBLE_DEVICES']='-1'
os.environ["KERAS_BACKEND"] = 'tensorflow'

from keras.models import Sequential
from keras.layers import Dense, LSTM

from finta import TA

In [ ]:
# Criando o modelo LSTM:
modelo = Sequential()
#modelo.add(LSTM(4, batch_input_shape = (2, x_treino.shape[1], x_treino.shape[2]), stateful = True))
modelo.add(LSTM(20, input_shape = (x_treino.shape[1], x_treino.shape[2]), dropout=0.05))


modelo.add(Dense(1, activation='relu')) # Saída da LSTM
modelo.compile(loss = 'mean_squared_error', optimizer = 'adam')
modelo.fit(x_treino, y_treino, epochs = 200, batch_size = 64, verbose = 2)

In [ ]:
modelo = Sequential()
#modelo.add(LSTM(4, batch_input_shape = (2, x_treino.shape[1], x_treino.shape[2]), stateful = True))
modelo.add(LSTM(20, input_shape = (x_treino.shape[1], x_treino.shape[2]), dropout=0.05))


modelo.add(Dense(1, activation='relu')) # Saída da LSTM
modelo.compile(loss = 'mean_squared_error', optimizer = 'adam')

In [ ]:
modelo.load_weights('save/lstm_price_predict_new.keras')

In [ ]:
y_treino.shape

In [ ]:
y_treino_dataset_size = np.zeros(shape=(y_treino.shape[0], 1))

In [ ]:
y_treino_dataset_size[:,-1] = y_treino

In [ ]:
#y_treino_dataset_size = y_treino
y_treino_dataset_size

In [ ]:
x_teste.shape

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [ ]:
#y_treino_dataset_size = y_treino
#y_treino_dataset_size = y_treino_dataset_size.reshape(1,-1)
y_treino = scaler.inverse_transform(y_treino_dataset_size)

In [ ]:
from sklearn.metrics import mean_squared_error


index_col = dataset.columns.get_loc('diff_target')
# Fazendo as previsões e avaliando a performance
previsoes_treino = modelo.predict(x_treino)
# A previsão tem como resultado um array de dimensões (n_amostras, 1), então se tentarmos executar o inverso da normalização
# diretamente com esses dados, teremos uma mensagem de erro, afinal a normalização foi aplicada em um dataset de dimensões:
# (n_amostras, 79), ou seja, a função scaler.inverse_transform não saberá o que fazer para gerar o inverso das outras 78 colunas.
# Portanto iremos primeiro gerar uma matriz com a mesma quantidade de colunas de x_treino (preenchida com zeros), depois iremos
# colocar o array de previsões em uma coluna dessa matriz (no lugar onde estava o price_actual), então poderemos aplicar a função
# scaler.inverse_transform e depois coletar somente a coluna que nos interessa.
# Obs: para descobrir qual o índice da coluna que possui a feature "price actual", basta utilizar o código: 
# treino.columns.get_loc("price actual")

# Previsões com os dados de treino:
previsoes_treino_dataset_size = np.zeros(shape=(len(previsoes_treino), x_treino.shape[2] +1))
previsoes_treino_dataset_size[:,index_col] = previsoes_treino[:,0]
previsoes_treino = scaler.inverse_transform(previsoes_treino_dataset_size)[:,index_col]

# Previsões com os dados de teste:
previsoes_teste = modelo.predict(x_teste)
previsoes_teste_dataset_size = np.zeros(shape=(len(previsoes_teste), x_teste.shape[2] +1))
previsoes_teste_dataset_size[:,index_col] = previsoes_teste[:,0]
previsoes_teste = scaler.inverse_transform(previsoes_teste_dataset_size)[:,index_col]

# Invertendo a normalização das variáveis target:
y_treino_dataset_size = np.zeros(shape=(len(y_treino), x_treino.shape[2] +1))
y_treino_dataset_size[:,index_col] = y_treino
y_treino = scaler.inverse_transform(y_treino_dataset_size)[:,index_col]

y_teste_dataset_size = np.zeros(shape=(len(y_teste), x_teste.shape[2] +1))
y_teste_dataset_size[:,index_col] = y_teste
y_teste = scaler.inverse_transform(y_teste_dataset_size)[:,index_col]

# Calculando o RMSE:
score_treino = math.sqrt(mean_squared_error(y_treino, previsoes_treino))
print('Score em Treino: %.2f RMSE' % (score_treino))  
score_teste = math.sqrt(mean_squared_error(y_teste, previsoes_teste))
print('Score em Teste: %.2f RMSE' % (score_teste))

In [ ]:
y_treino.shape

In [ ]:
# Mostrando as previsões do modelo:
# previsto = teste[tamanho_janela:].copy()  
# previsto['Close'] = previsoes_teste # coloca os valores das previsoes do modelo dentro dessa variável
fig, ax = plt.subplots(1, figsize=(12, 6))
# treino['Close'] = scaler.inverse_transform(treino['Close'].values.reshape(-1,1))
# teste['Close'] = scaler.inverse_transform(teste['Close'].values.reshape(-1,1))
#ax.plot(y_treino, label='treino', linewidth=2)
ax.plot(y_teste[101:500], label='teste', linewidth=2)
ax.plot(previsoes_teste[101:500], label='previsões', linewidth=1)
ax.set_ylabel('Função', fontsize=14)
ax.legend(loc='best', fontsize=16)

In [ ]:
previsoes_teste

### RL Trader - Course Script

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools

/tmp/ipykernel_6355/3301107732.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
df0 = pd.read_csv('data/ada_prices.csv', index_col=0, parse_dates=True)
#df0.dropna(axis=0, how='all', inplace=True)
#df0.dropna(axis=1, how='any', inplace=True)

In [4]:
df0

,Open,High,Low,Close,Volume,EMA,RSI,MACD,BB_UPPER,BB_MIDDLE,BB_LOWER
closetime,,,,,,,,,,,
2020-02-01 03:59:59.999,0.05516,0.05538,0.05457,0.05482,13425624.0,0.054363,61.190849,0.000228,0.055416,0.053853,0.052291
2020-02-01 04:59:59.999,0.05483,0.05540,0.05469,0.05495,13448954.0,0.054481,62.805730,0.000254,0.055479,0.053872,0.052265
2020-02-01 05:59:59.999,0.05492,0.05508,0.05476,0.05480,11942184.0,0.054546,59.587184,0.000262,0.055490,0.053875,0.052261
2020-02-01 06:59:59.999,0.05483,0.05506,0.05474,0.05478,11621543.0,0.054593,59.132637,0.000263,0.055553,0.053899,0.052244
2020-02-01 07:59:59.999,0.05478,0.05523,0.05472,0.05492,9376922.0,0.054658,61.448525,0.000271,0.055651,0.053937,0.052224
...,...,...,...,...,...,...,...,...,...,...,...
2024-03-20 19:59:59.999,0.62350,0.63480,0.61960,0.63210,55752143.0,0.611113,66.441848,0.000264,0.627959,0.602170,0.576381
2024-03-20 20:59:59.999,0.63220,0.64390,0.62910,0.63720,51886076.0,0.616330,68.621508,0.002473,0.634163,0.604035,0.573907
2024-03-20 21:59:59.999,0.63720,0.64330,0.63530,0.64050,19606479.0,0.621164,70.038137,0.004439,0.639976,0.606595,0.573214


In [ ]:
data = {}
for name in df0.columns:
  data[name] = np.log(df0[name]).diff()
df_returns = pd.DataFrame(data=data)

In [ ]:
# split into train and test
Ntest = 1000
train_data = df_returns.iloc[:-Ntest]
test_data = df_returns.iloc[-Ntest:]

In [ ]:
feats = ['AAPL', 'MSFT', 'AMZN']

In [ ]:
class Env:
  def __init__(self, df):
    self.df = df
    self.n = len(df)
    self.current_idx = 0
    self.action_space = [0, 1, 2] # BUY, SELL, HOLD
    self.invested = 0

    self.states = self.df[feats].to_numpy()
    self.rewards = self.df['SPY'].to_numpy()

  def reset(self):
    self.current_idx = 0
    return self.states[self.current_idx]

  def step(self, action):
    # need to return (next_state, reward, done)

    self.current_idx += 1
    if self.current_idx >= self.n:
      raise Exception("Episode already done")

    if action == 0: # BUY
      self.invested = 1
    elif action == 1: # SELL
      self.invested = 0
    
    # compute reward
    if self.invested:
      reward = self.rewards[self.current_idx]
    else:
      reward = 0

    # state transition
    next_state = self.states[self.current_idx]

    done = (self.current_idx == self.n - 1)
    return next_state, reward, done

In [ ]:
class StateMapper:
  def __init__(self, env, n_bins=6, n_samples=10000):
    # first, collect sample states from the environment
    states = []
    done = False
    s = env.reset()
    self.D = len(s) # number of elements we need to bin
    states.append(s)
    while True:
      a = np.random.choice(env.action_space)
      s2, _, done = env.step(a)
      states.append(s2)
      if len(states) >= n_samples:
        break
      if done:
        s = env.reset()
        states.append(s)
        if len(states) >= n_samples:
          break

    # convert to numpy array for easy indexing
    states = np.array(states)

    # create the bins for each dimension
    self.bins = []
    for d in range(self.D):
      column = np.sort(states[:,d])

      # find the boundaries for each bin
      current_bin = []
      for k in range(n_bins):
        boundary = column[int(n_samples / n_bins * (k + 0.5))]
        current_bin.append(boundary)

      self.bins.append(current_bin)


  def transform(self, state):
    x = np.zeros(self.D)
    for d in range(self.D):
      x[d] = int(np.digitize(state[d], self.bins[d]))
    return tuple(x)


  def all_possible_states(self):
    list_of_bins = []
    for d in range(self.D):
      list_of_bins.append(list(range(len(self.bins[d]) + 1)))
    # print(list_of_bins)
    return itertools.product(*list_of_bins)

In [ ]:
class Agent:
  def __init__(self, action_size, state_mapper):
    self.action_size = action_size
    self.gamma = 0.8  # discount rate
    self.epsilon = 0.1
    self.learning_rate = 1e-1
    self.state_mapper = state_mapper

    # initialize Q-table randomly
    self.Q = {}
    for s in self.state_mapper.all_possible_states():
      s = tuple(s)
      for a in range(self.action_size):
        self.Q[(s,a)] = np.random.randn()

  def act(self, state):
    if np.random.rand() <= self.epsilon:
      return np.random.choice(self.action_size)

    s = self.state_mapper.transform(state)
    act_values = [self.Q[(s,a)] for a in range(self.action_size)]
    return np.argmax(act_values)  # returns action

  def train(self, state, action, reward, next_state, done):
    s = self.state_mapper.transform(state)
    s2 = self.state_mapper.transform(next_state)

    if done:
      target = reward
    else:
      act_values = [self.Q[(s2,a)] for a in range(self.action_size)]
      target = reward + self.gamma * np.amax(act_values)

    # Run one training step
    self.Q[(s,action)] += self.learning_rate * (target - self.Q[(s,action)])

In [ ]:
def play_one_episode(agent, env, is_train):
  state = env.reset()
  done = False
  total_reward = 0

  while not done:
    action = agent.act(state)
    next_state, reward, done = env.step(action)
    total_reward += reward
    if is_train:
      agent.train(state, action, reward, next_state, done)
    state = next_state

  return total_reward

In [ ]:
num_episodes = 500

In [ ]:
train_env = Env(train_data)
test_env = Env(test_data)

In [ ]:
action_size = len(train_env.action_space)
state_mapper = StateMapper(train_env)
agent = Agent(action_size, state_mapper)
train_rewards = np.empty(num_episodes)
test_rewards = np.empty(num_episodes)

In [ ]:
for e in range(num_episodes):
  r = play_one_episode(agent, train_env, is_train=True)
  train_rewards[e] = r

  # test on the test set
  tmp_epsilon = agent.epsilon
  agent.epsilon = 0.
  tr = play_one_episode(agent, test_env, is_train=False)
  agent.epsilon = tmp_epsilon
  test_rewards[e] = tr

  print(f"eps: {e + 1}/{num_episodes}, train: {r:.5f}, test: {tr:.5f}")